In [ ]:
library(tidyr)
library(dplyr)
library(lubridate)
library(xts)

completeHvalerFile = "Hvaler/complete.csv"
completeHvalerClasses = c('POSIXct', rep("numeric", 21))
HvalerDf = read.csv(completeHvalerFile, stringsAsFactors=FALSE, colClasses=completeHvalerClasses)
xt = xts(x = HvalerDf[, -1], order.by = HvalerDf[, 1])

In [ ]:
#Impute temperature first (Hvaler has 9 missing values on temperature)
require("imputeTS")
imputed_xt = na.kalman(as.ts(xt$T01), model = "auto.arima", smooth = TRUE)
imputed_xts = xts(imputed_xt, index(xt))
HvalerDf[match(index(imputed_xts), HvalerDf$DateTime), "T01"] = coredata(imputed_xts)

In [ ]:
#str(imputedDf)
#imputed_DF = rfImpute(T01 ~ year + month + weekend + hour+ subs.1, imputedDf, ntree = 100)
library(Amelia)
df = HvalerDf %>% mutate(hour_weekend = as.factor(paste0(as.character(hour(DateTime)),
                                                         ".",
                                                         as.character(wday(DateTime) %in% c(1, 7)))))
cols = seq(2, 21)
min = rep(1, 20)
max = rep(9999999, 20)
bounds = as.matrix(cbind(cols, min, max))

out = amelia(df, ts = "T01", polytime = 2, cs = 'hour_weekend', intercs = TRUE, bounds=bounds, max.resample = 1000)

In [ ]:
imputedHvalerDf = out$imputations$imp1 %>% select (-hour_weekend)

for(colname in names(imputedHvalerDf)){
    if (colname != "DateTime"){
        x = imputedHvalerDf[[colname]]
        lag1week = c(rep(NA, 168), head(x, -168));
        coff = x/lag1week;
        problemIdx = sort(c(which(coff>=4),which(coff <=0.3))) #5 times than last week, it's a problem, mark as NA 
        imputedHvalerDf[[colname]][problemIdx] = imputedHvalerDf[[colname]][problemIdx-168] #use last week value 
    }
}
write.csv(imputedHvalerDf, "Hvaler/imputed_complete.csv", row.names=FALSE)

In [ ]:
#Draw imputed dataset
source("Lib/strip.R")

for (i in seq(1, 20)){
    zone = paste0("subs.", i);
    pdf(paste0("imputed_", zone, "_Hvaler_strip.pdf"),width=7,height=5)
    zoneDf = out$imputations$imp1
    strip(x = zoneDf[[zone]], 
      date = zoneDf$DateTime,
      cond = year(zoneDf$DateTime),
      arrange = "wide",
      main = paste("Consumption", zone))
    dev.off()
}